Lets Test it Buddy

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mlt
import os
import cv2
import sys
import mediapipe as mp
import math
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

Load up the Data

In [2]:
df = pd.read_csv('test.csv')

Angle_Calculator FN

In [3]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle

My isClose FN

In [4]:
def isclose(a, b, rel_tol=1e-09, abs_tol=0.6):
    return abs(a-b) <= max(rel_tol * max(abs(a), abs(b)), abs_tol)

New Match FN

In [5]:
def letsMatchIt(posList1, posList2):
  c4list = 0
  firstVectyXY = []
  secondVectyXY = []
  for element in range(len(posList1)):
      if c4list < 2:
          firstVectyXY.append(posList1[element])
          secondVectyXY.append(posList2[element])
      if c4list < 3:
          c4list += 1
      else: 
          c4list = 0
  

  # firstVectyXY = firstVectyXY[:-1]
  firstVectyConfidence = posList1[3::4]
  firstVectyConfidenceSum = sum(firstVectyConfidence)
  # Part1
  part1 = 1 / firstVectyConfidenceSum

  # Part2
  part2 = 0

  for i in range(len(firstVectyXY)):
    tempConf = math.floor(i/2)
    tempSum = firstVectyConfidence[tempConf] * abs(firstVectyXY[i] - secondVectyXY[i])
    part2 = part2 + tempSum

  return part1 * part2

In [6]:
landmarkz2 = ['x1',
 'y1',
 'z1',
 'v1',
 'x2',
 'y2',
 'z2',
 'v2',
 'x3',
 'y3',
 'z3',
 'v3',
 'x4',
 'y4',
 'z4',
 'v4',
 'x5',
 'y5',
 'z5',
 'v5',
 'x6',
 'y6',
 'z6',
 'v6',
 'x7',
 'y7',
 'z7',
 'v7',
 'x8',
 'y8',
 'z8',
 'v8',
 'x9',
 'y9',
 'z9',
 'v9',
 'x10',
 'y10',
 'z10',
 'v10',
 'x11',
 'y11',
 'z11',
 'v11',
 'x12',
 'y12',
 'z12',
 'v12',
 'x13',
 'y13',
 'z13',
 'v13',
 'x14',
 'y14',
 'z14',
 'v14',
 'x15',
 'y15',
 'z15',
 'v15',
 'x16',
 'y16',
 'z16',
 'v16',
 'x17',
 'y17',
 'z17',
 'v17',
 'x18',
 'y18',
 'z18',
 'v18',
 'x19',
 'y19',
 'z19',
 'v19',
 'x20',
 'y20',
 'z20',
 'v20',
 'x21',
 'y21',
 'z21',
 'v21',
 'x22',
 'y22',
 'z22',
 'v22',
 'x23',
 'y23',
 'z23',
 'v23',
 'x24',
 'y24',
 'z24',
 'v24',
 'x25',
 'y25',
 'z25',
 'v25',
 'x26',
 'y26',
 'z26',
 'v26',
 'x27',
 'y27',
 'z27',
 'v27',
 'x28',
 'y28',
 'z28',
 'v28',
 'x29',
 'y29',
 'z29',
 'v29',
 'x30',
 'y30',
 'z30',
 'v30',
 'x31',
 'y31',
 'z31',
 'v31',
 'x32',
 'y32',
 'z32',
 'v32',
 'x33',
 'y33',
 'z33',
 'v33',
 'a11',
 'a12',
 'a13',
 'a14',
 'a16',
 'a17',
 'a23',
 'a24',
 'a25',
 'a26',
 'a27',
 'a28',
 'path',]

In [7]:
cap = cv2.VideoCapture('n_vid2.mp4')
final_list = []
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.9, min_tracking_confidence=0.9) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        # re_frame = cv2.resize(frame, (608, 1080))
        if ret == True:
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
        
            # Make detection
            results = pose.process(image)
        
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # Extract landmarks
            try:
                #The Marks
                landmarks = results.pose_landmarks.landmark
                
                if len(landmarks) != 33:
                    pass

                l_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                l_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                l_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                l_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                l_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                l_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                l_heel = [landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y]
                
                r_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                r_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                r_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                r_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                r_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
                r_heel = [landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].y]


                #Angle Calculate
                # a11 = calculate_angle(l_elbow,l_shoulder,l_hip)
                # a12 = calculate_angle(r_elbow,r_shoulder,r_hip)
                # a13 = calculate_angle(l_shoulder,l_elbow, l_wrist)
                # a14 = calculate_angle(r_shoulder,r_elbow, r_wrist)
                # a23 = calculate_angle(l_shoulder,l_hip,l_knee)
                # a24 = calculate_angle(r_shoulder,r_hip,r_knee)
                # a25 = calculate_angle(l_hip,l_knee,l_ankle)
                # a26 = calculate_angle(r_hip,r_knee,r_ankle)
                # a27 = calculate_angle(l_knee,l_ankle,l_heel)
                # a28 = calculate_angle(r_knee,r_ankle,r_heel)



                # found = df.loc[np.isclose(df['a11'],a11,1e-09,0.06) | np.isclose(df['a12'],a12,1e-09,0.06) | np.isclose(df['a13'],a13,1e-09,0.06) | np.isclose(df['a14'],a14,1e-09,0.06) | np.isclose(df['a23'],a23,1e-09,0.06) | np.isclose(df['a24'],a24,1e-09,0.06) | np.isclose(df['a25'],a25,1e-09,0.06) | np.isclose(df['a26'],a26,1e-09,0.06) | np.isclose(df['a27'],a27,1e-09,0.06) | np.isclose(df['a28'],a28,1e-09,0.06)]

                # if found.empty:
                #     continue
                # else:
                #     # break
                #     final_list.append(list(x for x in found["path"]))
                #     # final_list.append(df["path"].values.tolist())
                #     print(found["path"])
                firstVecty = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
                for i in range(len(df)):
                    secondVecty = df.iloc[i].values.flatten().tolist()
                    theMatch = letsMatchIt(firstVecty,secondVecty)
                    if(theMatch < 0.06):
                        final_list.append(secondVecty[-1])
                        break

            except:
                pass


            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        else:
            break

    cap.release()
    cv2.destroyAllWindows()

Create Video From Found List

In [3]:
final_list = ['C:/Users/dedsec995/shakeit/data/image13.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image12.jpg', 'C:/Users/dedsec995/shakeit/data/image13.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image11.jpg', 'C:/Users/dedsec995/shakeit/data/image12.jpg', 'C:/Users/dedsec995/shakeit/data/image12.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image13.jpg', 'C:/Users/dedsec995/shakeit/data/image29.jpg', 'C:/Users/dedsec995/shakeit/data/image13.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image12.jpg', 'C:/Users/dedsec995/shakeit/data/image12.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image40.jpg', 'C:/Users/dedsec995/shakeit/data/image40.jpg', 'C:/Users/dedsec995/shakeit/data/image13.jpg', 'C:/Users/dedsec995/shakeit/data/image13.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image10.jpg', 'C:/Users/dedsec995/shakeit/data/image10.jpg', 'C:/Users/dedsec995/shakeit/data/image8.jpg', 'C:/Users/dedsec995/shakeit/data/image4.jpg', 'C:/Users/dedsec995/shakeit/data/image2.jpg', 'C:/Users/dedsec995/shakeit/data/image2.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image13.jpg', 'C:/Users/dedsec995/shakeit/data/image13.jpg', 'C:/Users/dedsec995/shakeit/data/image30.jpg', 'C:/Users/dedsec995/shakeit/data/image30.jpg', 'C:/Users/dedsec995/shakeit/data/image30.jpg', 'C:/Users/dedsec995/shakeit/data/image20.jpg', 'C:/Users/dedsec995/shakeit/data/image20.jpg', 'C:/Users/dedsec995/shakeit/data/image20.jpg', 'C:/Users/dedsec995/shakeit/data/image20.jpg', 'C:/Users/dedsec995/shakeit/data/image20.jpg', 'C:/Users/dedsec995/shakeit/data/image20.jpg', 'C:/Users/dedsec995/shakeit/data/image21.jpg', 'C:/Users/dedsec995/shakeit/data/image21.jpg', 'C:/Users/dedsec995/shakeit/data/image19.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image1.jpg', 'C:/Users/dedsec995/shakeit/data/image8.jpg', 'C:/Users/dedsec995/shakeit/data/image41.jpg', 'C:/Users/dedsec995/shakeit/data/image13.jpg', 'C:/Users/dedsec995/shakeit/data/image13.jpg', 'C:/Users/dedsec995/shakeit/data/image13.jpg', 'C:/Users/dedsec995/shakeit/data/image13.jpg', 'C:/Users/dedsec995/shakeit/data/image13.jpg', 'C:/Users/dedsec995/shakeit/data/image13.jpg', 'C:/Users/dedsec995/shakeit/data/image11.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg']
video_name = 'video.mp4'
images = [img for img in final_list]
# print(images)
# print(type(images))
# print(images[0])
# print(type(images[0][0]))
frame = cv2.imread(images[0])
height, width, layers = frame.shape
video = cv2.VideoWriter(video_name, 0, 14, (width,height))
for image in images:
    # cv2.imshow(image[0])
    video.write(cv2.imread(image))

cv2.destroyAllWindows()
video.release()

Create Video from Folder

In [21]:
image_folder = 'data'
video_name = 'video.avi'

images = [img for img in os.listdir(image_folder) if img.endswith(".jpg")]
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, 0, 14, (width,height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()

In [56]:
display(found['path'])
values = list(x for x in found["path"])
print(values)

15    C:\Users\dedsec995\shakeit\data\image16.jpg
Name: path, dtype: object

[['C:\\Users\\dedsec995\\shakeit\\data\\image16.jpg']]


In [9]:
print(final_list)

['C:/Users/dedsec995/shakeit/data/image13.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image12.jpg', 'C:/Users/dedsec995/shakeit/data/image13.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image11.jpg', 'C:/Users/dedsec995/shakeit/data/image12.jpg', 'C:/Users/dedsec995/shakeit/data/image12.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit/data/image9.jpg', 'C:/Users/dedsec995/shakeit